# PySpark cache danger

In [1]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [2]:
import pyspark
from delta import *
from pyspark.sql import functions as F

builder = (
    pyspark.sql.SparkSession.builder.appName("MyApp")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config(
        "spark.sql.catalog.spark_catalog",
        "org.apache.spark.sql.delta.catalog.DeltaCatalog",
    )
)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

24/09/30 16:32:47 WARN Utils: Your hostname, Matthews-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.2.211.112 instead (on interface en0)
24/09/30 16:32:47 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/matthewpowers/.ivy2/cache
The jars for the packages stored in: /Users/matthewpowers/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c39964da-f902-4fa6-a88a-fc1f548aaf55;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.2.0 in central
	found io.delta#delta-storage;3.2.0 in central


:: loading settings :: url = jar:file:/opt/miniconda3/envs/pyspark-350-delta-320/lib/python3.11/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.antlr#antlr4-runtime;4.9.3 in central
:: resolution report :: resolve 80ms :: artifacts dl 3ms
	:: modules in use:
	io.delta#delta-spark_2.12;3.2.0 from central in [default]
	io.delta#delta-storage;3.2.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-c39964da-f902-4fa6-a88a-fc1f548aaf55
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/2ms)
24/09/30 16:32:47 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-ja

In [19]:
df = spark.createDataFrame([(0, "Bob", 23), (1, "Sue", 25), (2, "Jim", 27)]).toDF(
    "id", "name", "age"
)

In [20]:
df.repartition(1).write.mode("append").format("delta").saveAsTable("students")

In [21]:
spark.table("students").show()

+---+----+---+
| id|name|age|
+---+----+---+
|  0| Bob| 23|
|  1| Sue| 25|
|  2| Jim| 27|
+---+----+---+



In [22]:
df = spark.table("students").cache()

In [23]:
df.count()

3

In [24]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  0| Bob| 23|
|  1| Sue| 25|
|  2| Jim| 27|
+---+----+---+



In [25]:
from delta import DeltaTable
from pyspark.sql.functions import col

In [26]:
dt = DeltaTable.forName(spark, "students")
dt.delete(F.col("age") > 24)

In [27]:
df.show()

+---+----+---+
| id|name|age|
+---+----+---+
|  0| Bob| 23|
+---+----+---+



In [16]:
spark.sql("drop table students")

DataFrame[]

In [17]:
spark.table("students").show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `students` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.;
'UnresolvedRelation [students], [], false
